In [ ]:
import tens

In [ ]:
from selenium.webdriver import Chrome,ChromeOptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import cv2
import mediapipe as mp
import time
import threading
import queue
import math
import tkinter as tk
from PIL import Image, ImageTk

In [ ]:
options = ChromeOptions()

options.add_argument(r"user-data-dir=C:\Users\abhay\Desktop\CG (CMP513, CMP514)\Project - COPY\browser")
# options.add_argument("--headless")
options.add_argument('--log-level=1')

browser = Chrome(options=options)
wait = WebDriverWait(browser, 3)
action = ActionChains(browser)


browser.get("https://open.spotify.com/")

try:
    if wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "ButtonInner-sc-14ud5tc-0.kwEVAz.encore-inverted-light-set"))).text == "Log in":
        logged_in = False
except:
    logged_in = True

if not logged_in:
    print("User is not logged in. Restarting in non-headless mode.")
    browser.quit()

    options = ChromeOptions()
    options.add_argument(r"user-data-dir=C:\Users\abhay\Desktop\CG (CMP513, CMP514)\Project - COPY\browser")

    browser = Chrome(options=options)
    browser.get(r"https://accounts.spotify.com/en/login")

    br = True
    while(br):
        try: 
            for text in browser.find_elements(By.CLASS_NAME, "encore-text.encore-text-body-medium-bold.sc-iKTcqh.doOTMr"):
                if text.text == "Web Player":
                    br = False
        except:
            time.sleep(2)
    browser.quit()

    options = ChromeOptions()
    options.add_argument(r"user-data-dir=C:\Users\abhay\Desktop\CG (CMP513, CMP514)\Project - COPY\browser")
    options.add_argument("--headless")

    browser = Chrome(options=options)
    wait = WebDriverWait(browser, 10)
    browser.get(r"https://open.spotify.com/")
else:
    print("User is already logged in. Continuing in headless mode.")

main_hub = browser.find_element(By.CLASS_NAME, "yglmI5m3fCc8baD1Kwdw")

In [ ]:
def is_only_index_and_middle_extended(landmarks):
    index_extended = landmarks[8].y < landmarks[6].y 
    middle_extended = landmarks[12].y < landmarks[10].y
    ring_closed = landmarks[16].y > landmarks[14].y 
    pinky_closed = landmarks[20].y > landmarks[18].y
    
    return index_extended and middle_extended and ring_closed and pinky_closed

def is_only_thumb_and_index_extended(landmarks):
    thumb_extended = landmarks[4].x > landmarks[3].x 
    index_extended = landmarks[8].y < landmarks[6].y
    middle_closed = landmarks[11].y > landmarks[9].y
    ring_closed = landmarks[15].y > landmarks[13].y
    pinky_closed = landmarks[19].y > landmarks[17].y

    return thumb_extended and index_extended and middle_closed and ring_closed and pinky_closed

def get_thumb_index_distance_action(landmarks, last_volume_time, min_distance=0.1, max_distance=0.21, cooldown = 0.7):
    thumb_x, thumb_y = landmarks[4].x, landmarks[4].y
    index_x, index_y = landmarks[8].x, landmarks[8].y
    
    # Calculate Euclidean distance between thumb and index finger
    distance = math.sqrt((index_x - thumb_x) ** 2 + (index_y - thumb_y) ** 2)

    current_time = time.time()
    if current_time - last_volume_time < cooldown:
        return False
    
    # Determine action based on the distance
    if distance > max_distance:
        last_volume_time = current_time
        return "volume up"
    elif distance < min_distance:
        return "volume down"
    else:
        return False

def is_hand_still(current_position, last_position, last_movement_time, still_threshold=0.01, still_duration=1):
    distance = ((current_position[0] - last_position[0]) ** 2 + (current_position[1] - last_position[1]) ** 2) ** 0.5
    
    current_time = time.time()

    if distance < still_threshold:
        if current_time - last_movement_time >= still_duration:
            return True, current_time
    else:
        last_movement_time = current_time

    return False, last_movement_time

def detect_swipe(previous_position, current_position, swipe_threshold=0.08):
    distance_moved = abs(current_position[0] - previous_position[0])

    if distance_moved > swipe_threshold:
        swipe_direction = 'left' if current_position[0] < previous_position[0] else 'right'
        return swipe_direction
    return None

def is_fist_closed(landmarks, right_hand):
    # Check if the hand is closed into a fist AND if it is facing the camera
    return (landmarks[8].y > landmarks[7].y > landmarks[6].y and 
            landmarks[12].y > landmarks[11].y > landmarks[9].y and
            landmarks[16].y > landmarks[15].y > landmarks[13].y and
            landmarks[20].y > landmarks[19].y > landmarks[17].y and
            landmarks[4].x < landmarks[3].x and
            is_facing_camera(landmarks, right_hand))

def is_palm_open(landmarks, right_hand):
    # Check if the hand is open, palm facing outward AND if it is facing the camera
    return (landmarks[4].y < landmarks[3].y < landmarks[2].y < landmarks[1].y and   # Thumb
            landmarks[8].y < landmarks[7].y < landmarks[6].y < landmarks[5].y and   # Index finger
            landmarks[12].y < landmarks[11].y < landmarks[10].y < landmarks[9].y and  # Middle finger
            landmarks[16].y < landmarks[15].y < landmarks[14].y < landmarks[13].y and # Ring finger
            landmarks[20].y < landmarks[19].y < landmarks[18].y < landmarks[17].y and # Pinky finger
            is_facing_camera(landmarks, right_hand))

def is_facing_camera(landmarks, right_hand):    
    if right_hand:
        return (landmarks[5].x > landmarks[9].x > landmarks[13].x > landmarks[17].x)
    else:  # Left hand
        return (landmarks[5].x < landmarks[9].x < landmarks[13].x < landmarks[17].x)

def send_command(command):
    try:
        main_hub.find_element(By.CLASS_NAME, "gQoa8JTSpjSmYyABcag2.encore-bright-accent-set.T3hkVxXuSbCYOD2GIeQd")
    except:
        return "No Devices Available"
    
    if command == "play":
        if main_hub.find_element(By.CLASS_NAME, "Button-sc-qlcn5g-0.iJUiBm").get_attribute("aria-label") == "Play":
            main_hub.find_element(By.CLASS_NAME, "Button-sc-qlcn5g-0.iJUiBm").click()
            return "Play"
        else:
            return "Device is already playing."
    
    elif command == "pause":
        if main_hub.find_element(By.CLASS_NAME, "Button-sc-qlcn5g-0.iJUiBm").get_attribute("aria-label") == "Pause":
            main_hub.find_element(By.CLASS_NAME, "Button-sc-qlcn5g-0.iJUiBm").click()
            return "Pause"
        else:
            return "Device is already pasued."

    elif command == "next_track":
        side_hub = main_hub.find_element(By.CLASS_NAME, "Qt226Z4rBQs53aedRQBQ")
        for button in side_hub.find_elements(By.CLASS_NAME, "Button-sc-1dqy6lx-0.dmdXQN"):
            if button.get_attribute("data-testid") == "control-button-skip-forward":
                button.click()
                return "Next track"

    elif command == "prev_track":
        song_name = main_hub.find_element(By.XPATH, "//*[@id=\"main\"]/div/div[2]/div[3]/footer/div[1]/div[1]/div/div[2]/div[1]/div/div/div/div/span/a").text
        side_hub = main_hub.find_element(By.CLASS_NAME, "NKUrT1GciYXAEEUtagN1")
        for button in side_hub.find_elements(By.CLASS_NAME, "Button-sc-1dqy6lx-0.dmdXQN"):
            if button.get_attribute("data-testid") == "control-button-skip-back":
                button.click()
                try:
                    WebDriverWait(browser, 0.8).until(EC.element_to_be_clickable(("")))
                except:
                    if song_name == main_hub.find_element(By.XPATH, "//*[@id=\"main\"]/div/div[2]/div[3]/footer/div[1]/div[1]/div/div[2]/div[1]/div/div/div/div/span/a").text:
                        button.click()
                    return "Previous track"
    
    elif command == "volume_up":
        value = float(main_hub.find_element(By.CLASS_NAME, "G4n5bTzWUvlftzDwrFVG.ExuDUBJ7bk8vT6INnm9F").find_element(By.TAG_NAME, "input").get_attribute("value"))
        if value == 1:
            return "Max Volume"
        value+=0.1
        slider = main_hub.find_element(By.CLASS_NAME, "G4n5bTzWUvlftzDwrFVG.ExuDUBJ7bk8vT6INnm9F").find_element(By.CLASS_NAME, "tIr7C6B0Pt6YKdOnqaqj")
        slider_size = slider.size
        action.move_to_element_with_offset(slider, (0 - slider_size['width'] / 2) + slider_size['width'] * value, 0).click().perform()
        return f"Volume: {str(value*100)[0:2]}%"
        
    elif command == "volume_down":
        value = float(main_hub.find_element(By.CLASS_NAME, "G4n5bTzWUvlftzDwrFVG.ExuDUBJ7bk8vT6INnm9F").find_element(By.TAG_NAME, "input").get_attribute("value"))
        if value == 0:
            return "Min Volume"
        value-=0.1
        slider = main_hub.find_element(By.CLASS_NAME, "G4n5bTzWUvlftzDwrFVG.ExuDUBJ7bk8vT6INnm9F").find_element(By.CLASS_NAME, "tIr7C6B0Pt6YKdOnqaqj")
        slider_size = slider.size
        action.move_to_element_with_offset(slider, (0 - slider_size['width'] / 2) + slider_size['width'] * value, 0).click().perform()
        return f"Volume: {str(value*100)[0:2]}%"
        

command_queue = queue.Queue()

def send_command_thread(command):
    # Worker function to execute command and put the result in the queue
    def worker():
        result = send_command(command)
        command_queue.put(result)
        
    
    # Start the thread
    thread = threading.Thread(target=worker)
    thread.start()
    return thread


In [ ]:
cap = cv2.VideoCapture(1)

In [ ]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

previous_position = None
last_movement_time = time.time()
last_volume_time = time.time()
hand_ready_for_swipe = False
last_fist_time = time.time()
last_palm_time = time.time()
fist_detected = False
palm_detected = False
hand_ready_for_gesture = False

song_name = main_hub.find_element(By.XPATH, "//*[@id=\"main\"]/div/div[2]/div[3]/footer/div[1]/div[1]/div/div[2]/div[1]/div/div/div/div/span/a").text

while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue

    results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            index_finger_tip = hand_landmarks.landmark[12]
            current_position = (index_finger_tip.x, index_finger_tip.y)

            if previous_position is None:
                previous_position = current_position

            if is_only_index_and_middle_extended(hand_landmarks.landmark):
                hand_still, last_movement_time = is_hand_still(current_position, previous_position, last_movement_time,  still_duration=0.2)
                if hand_still:
                    hand_ready_for_swipe = True
                if hand_ready_for_swipe:
                    swipe_direction = detect_swipe(previous_position, current_position)
                    if swipe_direction == 'left':
                        last_movement_time = time.time()
                        send_command_thread("prev_track")
                        print("Swipe left detected: Previous track")
                        hand_ready_for_swipe = False
                        previous_position = current_position
                    elif swipe_direction == 'right':
                        last_movement_time = time.time()
                        send_command_thread("next_track")
                        print("Swipe right detected: Next track")
                        hand_ready_for_swipe = False
                        previous_position = current_position

            if is_fist_closed(hand_landmarks.landmark, results.multi_handedness[0].classification[0].label[0] == "L"):
                hand_still, last_movement_time = is_hand_still(current_position, previous_position, last_movement_time)
                
                if hand_still:
                    hand_ready_for_gesture = True
                if hand_ready_for_gesture and not fist_detected:
                    last_fist_time = time.time()
                    fist_detected = True
                elif fist_detected and time.time() - last_fist_time > 1:
                    last_movement_time = time.time()
                    send_command_thread("pause")
                    print("Fist closed: Pausing")
                    hand_ready_for_gesture = False
                    fist_detected = False
            else:
                fist_detected = False

            if is_palm_open(hand_landmarks.landmark, results.multi_handedness[0].classification[0].label[0] == "L"):
                hand_still, last_movement_time = is_hand_still(current_position, previous_position, last_movement_time)
                
                if hand_still:
                    hand_ready_for_gesture = True
                if hand_ready_for_gesture and not palm_detected:
                    last_palm_time = time.time()
                    palm_detected = True
                elif palm_detected and time.time() - last_palm_time > 1:
                    last_movement_time = time.time()
                    send_command_thread("play")
                    print("Open palm: Playing")
                    hand_ready_for_gesture = False
                    palm_detected = False
            else:
                palm_detected = False

            if is_only_thumb_and_index_extended(hand_landmarks.landmark):
                volume = get_thumb_index_distance_action(hand_landmarks.landmark, last_volume_time)
                if volume == "volume up":
                    last_volume_time = time.time()
                    send_command_thread("volume_up")
                elif volume == "volume down":
                    last_volume_time = time.time()
                    send_command_thread("volume_down")
            

            previous_position = current_position
    elif hand_ready_for_swipe or hand_ready_for_gesture:
        hand_ready_for_swipe = False
        hand_ready_for_gesture = False

    cv2.imshow('Hand Gesture Control', image)
    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

previous_position = None
last_movement_time = time.time()
last_volume_time = time.time()
hand_ready_for_swipe = False
last_fist_time = time.time()
last_palm_time = time.time()
fist_detected = False
palm_detected = False
hand_ready_for_gesture = False
command_text = None
command_display_time = 3
command_last_time = time.time() - command_display_time

song_name = main_hub.find_element(By.XPATH, "//*[@id=\"main\"]/div/div[2]/div[3]/footer/div[1]/div[1]/div/div[2]/div[1]/div/div/div/div/span/a").text

while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue

    results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Gesture detection logic
            index_finger_tip = hand_landmarks.landmark[12]
            current_position = (index_finger_tip.x, index_finger_tip.y)

            if previous_position is None:
                previous_position = current_position

            # Check for swipe gestures
            if is_only_index_and_middle_extended(hand_landmarks.landmark):
                hand_still, last_movement_time = is_hand_still(current_position, previous_position, last_movement_time, still_duration=0.2)
                if hand_still:
                    hand_ready_for_swipe = True
                if hand_ready_for_swipe:
                    swipe_direction  = detect_swipe(previous_position, current_position)
                    if swipe_direction == 'left':
                        last_movement_time = time.time()
                        send_command_thread("prev_track")
                        command_last_time = time.time()
                        hand_ready_for_swipe = False
                    elif swipe_direction == 'right':
                        last_movement_time = time.time()
                        send_command_thread("next_track")
                        command_last_time = time.time()
                        hand_ready_for_swipe = False
                        previous_position = current_position

            # Check for fist (pause) gesture
            if is_fist_closed(hand_landmarks.landmark, results.multi_handedness[0].classification[0].label[0] == "L"):
                hand_still, last_movement_time = is_hand_still(current_position, previous_position, last_movement_time)
                
                if hand_still:
                    hand_ready_for_gesture = True
                if hand_ready_for_gesture and not fist_detected:
                    last_fist_time = time.time()
                    fist_detected = True
                elif fist_detected and time.time() - last_fist_time > 1:
                    last_movement_time = time.time()
                    send_command_thread("pause")
                    command_last_time = time.time()
                    hand_ready_for_gesture = False
                    fist_detected = False
            else:
                fist_detected = False

            if is_palm_open(hand_landmarks.landmark, results.multi_handedness[0].classification[0].label[0] == "L"):
                hand_still, last_movement_time = is_hand_still(current_position, previous_position, last_movement_time)
                
                if hand_still:
                    hand_ready_for_gesture = True
                if hand_ready_for_gesture and not palm_detected:
                    last_palm_time = time.time()
                    palm_detected = True
                elif palm_detected and time.time() - last_palm_time > 1:
                    last_movement_time = time.time()
                    send_command_thread("play")
                    command_last_time = time.time()
                    hand_ready_for_gesture = False
                    palm_detected = False
            else:
                palm_detected = False

            if is_only_thumb_and_index_extended(hand_landmarks.landmark):
                volume = get_thumb_index_distance_action(hand_landmarks.landmark, last_volume_time)
                if volume == "volume up":
                    last_volume_time = time.time()
                    send_command_thread("volume_up")
                elif volume == "volume down":
                    last_volume_time = time.time()
                    send_command_thread("volume_down")

            previous_position = current_position
    elif hand_ready_for_swipe or hand_ready_for_gesture:
        hand_ready_for_swipe = False
        hand_ready_for_gesture = False

    # Display song name
    cv2.putText(image, f"Song: {song_name}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)

    if not command_queue.empty():
        command_text = command_queue.get()
        command_last_time = time.time()
        

    if time.time() - command_last_time < command_display_time:
        try:
            song_name_element = wait.until(EC.visibility_of_element_located(
            (By.XPATH, "//*[@id=\"main\"]/div/div[2]/div[3]/footer/div[1]/div[1]/div/div[2]/div[1]/div/div/div/div/span/a")
        ))
            song_name = song_name_element.text 
        finally:
            cv2.putText(image, command_text, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)
    else:
        command_text = None
    # Show the image in the window
    cv2.imshow('Hand Gesture Control', image)

    # Exit on pressing 'ESC'
    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

previous_position = None
last_movement_time = time.time()
last_volume_time = time.time()
hand_ready_for_swipe = False
last_fist_time = time.time()
last_palm_time = time.time()
fist_detected = False
palm_detected = False
hand_ready_for_gesture = False
command_text = None
command_display_time = 3
command_last_time = time.time() - command_display_time
song_name = ""

# Global variables
cap2 = None  # Second video capture
is_playing = False
volume = 50  # Initial volume level (0-100)
video_size = (300, 200)  # Desired size of the video (width, height)

# Dictionary of actions to their corresponding video files
video_files = {
    "play": r"Gesture_Video\play.mp4",  # Replace with your actual video paths
    "pause": r"Gesture_Video\Pause.mp4",
    "next": r"Gesture_Video\Next.mp4",
    "prev": r"Gesture_Video\previous.mp4",
    "volume_up": r"Gesture_Video\Volume up.mp4",
    "volume_down": r"Gesture_Video\Volume down.mp4",
}

def start_video(video_name):
    global cap2, is_playing
    if cap2 is not None:  # Only release if cap2 is already initialized
        cap2.release()  # Release any previously opened video capture
        is_playing = False
    cap2 = cv2.VideoCapture(video_files[video_name])
    if not cap2.isOpened():
        print(f"Error: Could not open video {video_files[video_name]}")
        return
    is_playing = True
    update_video2()

def update_video2():
    """Update the small window with the current frame of the video."""
    global cap2, is_playing
    if is_playing and cap2 is not None:
        ret, frame = cap2.read()
        if ret:
            # Resize the frame to the desired dimensions
            frame_resized = cv2.resize(frame, video_size)
            frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(frame_rgb)
            imgtk = ImageTk.PhotoImage(image=img)
            video_label2.imgtk = imgtk
            video_label2.config(image=imgtk)
        else:
            # Video has finished, dispose of it
            cap2.release()  # Release the video capture
            is_playing = False
    # Update every 30ms
    video_label2.after(60, update_video2)

def update_info_panel():
    """Update the information panel text."""
    global song_name
    try:
        song_name_element = wait.until(EC.visibility_of_element_located((By.XPATH, "//*[@id=\"main\"]/div/div[2]/div[3]/footer/div[1]/div[1]/div/div[2]/div[1]/div/div/div/div/span/a")))
        song_name = song_name_element.text
    except:
        song_name = "Error Fetching Song Name"
    finally:
        info_text.set(song_name)
        root.after(1000, update_info_panel)
    
def update_command():
    global command_last_time, command_text
    if not command_queue.empty():
        command_text = command_queue.get()
        command_last_time = time.time()

    if not time.time() - command_last_time < command_display_time:
        command_text = ""
    info_text2.set(command_text)
    root.after(500, update_command)

# Set up the main window
root = tk.Tk()
root.title("Spotify Gesture Control")
root.geometry("1200x500")

# Main webcam feed frame (left side)
video_frame = tk.Frame(root, width=600, height=600)
video_frame.pack(side="left", padx=10, pady=10)

video_label = tk.Label(video_frame)
video_label.pack()

cap = cv2.VideoCapture(1)

if not cap.isOpened():
    print("Error: Could not access the webcam.")
    exit()

# Info panel (right side)
info_frame = tk.Frame(root, width=400, height=150, bg="#f0f0f0")
info_frame.pack(side="top", fill="x", padx=10, pady=10)

# Song name label (larger font, bold, centered)
info_label = tk.Label(info_frame, text="Spotify Gesture Control", font=("Arial", 16, "bold"), bg="#f0f0f0")
info_label.pack(pady=5)

# Song name display
info_text = tk.StringVar()
info_text.set("Loading song...")  # Initial placeholder
song_label = tk.Label(info_frame, textvariable=info_text, font=("Arial", 18), bg="#f0f0f0", fg="#333")
song_label.pack(pady=5)

# Command performed display (smaller font, lighter color)
info_text2 = tk.StringVar()
info_text2.set("Awaiting gesture...")  # Initial placeholder for commands
command_label = tk.Label(info_frame, textvariable=info_text2, font=("Arial", 14), bg="#f0f0f0", fg="#666")
command_label.pack(pady=5)

# Control panel (bottom-right, video and buttons)
control_frame = tk.Frame(root, width=400, height=250, bg="#f2f2f2")
control_frame.pack(side="bottom", fill="x", padx=20, pady=20)

# Small video playback window (top-right of the control panel)
# Create the small_video_frame with fixed size and prevent resizing
small_video_frame = tk.Frame(control_frame, width=300, height=180, bg="#333333", relief="solid", borderwidth=2)
small_video_frame.pack(side="top", padx=20, pady=10)

# Prevent the frame from resizing to fit its contents
small_video_frame.pack_propagate(False)

# Add video label inside small_video_frame (will stay centered)
video_label2 = tk.Label(small_video_frame, fg="white", bg="white")
video_label2.pack(fill="both", expand=True)

# Create a label below the small_video_frame for the "View Gesture" text
view_gesture_label = tk.Label(control_frame, text="View Gesture", font=("Arial", 12))
view_gesture_label.pack(side="top", pady=5)


# Buttons for control (each button triggers its own video)
button_style = {'width': 12, 'height': 1, 'font': ('Arial', 12), 'relief': 'raised', 'bd': 2, 'bg': '#4CAF50', 'fg': 'white'}

# Buttons in the control panel
# Frame for the first row of buttons, centered
row1_frame = tk.Frame(control_frame)
row1_frame.pack(side="top", padx=10, pady=5, anchor="center")

# Add buttons to the first row
play_button = tk.Button(row1_frame, text="Play", command=lambda: start_video("play"), **button_style)
play_button.pack(side="left", padx=10)

next_button = tk.Button(row1_frame, text="Next Track", command=lambda: start_video("next"), **button_style)
next_button.pack(side="left", padx=10)

prev_button = tk.Button(row1_frame, text="Previous Track", command=lambda: start_video("prev"), **button_style)
prev_button.pack(side="left", padx=10)

# Frame for the second row of buttons
row2_frame = tk.Frame(control_frame)
row2_frame.pack(side="top", padx=10, pady=5, anchor="center")

pause_button = tk.Button(row2_frame, text="Pause", command=lambda: start_video("pause"), **button_style)
pause_button.pack(side="left", padx=10)

# Add buttons to the second row
volume_up_button = tk.Button(row2_frame, text="Volume Up", command=lambda: start_video("volume_up"), **button_style)
volume_up_button.pack(side="left", padx=10)

volume_down_button = tk.Button(row2_frame, text="Volume Down", command=lambda: start_video("volume_down"), **button_style)
volume_down_button.pack(side="left", padx=10)

# Start the main video stream (webcam)
def update_video():
    global previous_position, last_movement_time, last_volume_time, hand_ready_for_swipe, last_fist_time, last_palm_time, fist_detected, palm_detected, hand_ready_for_gesture, command_last_time
    success, image = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        return

    frame_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    results = hands.process(frame_rgb)
    # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame_rgb, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            index_finger_tip = hand_landmarks.landmark[12]
            current_position = (index_finger_tip.x, index_finger_tip.y)

            if previous_position is None:
                previous_position = current_position

            if is_only_index_and_middle_extended(hand_landmarks.landmark):
                hand_still, last_movement_time = is_hand_still(current_position, previous_position, last_movement_time,  still_duration=0.2)
                if hand_still:
                    hand_ready_for_swipe = True
                if hand_ready_for_swipe:
                    swipe_direction = detect_swipe(previous_position, current_position)
                    if swipe_direction == 'left':
                        last_movement_time = time.time()
                        send_command_thread("prev_track")
                        command_last_time = time.time()
                        print("Swipe left detected: Previous track")
                        hand_ready_for_swipe = False
                        previous_position = current_position
                    elif swipe_direction == 'right':
                        last_movement_time = time.time()
                        send_command_thread("next_track")
                        command_last_time = time.time()
                        print("Swipe right detected: Next track")
                        hand_ready_for_swipe = False
                        previous_position = current_position

            if is_fist_closed(hand_landmarks.landmark, results.multi_handedness[0].classification[0].label[0] == "L"):
                hand_still, last_movement_time = is_hand_still(current_position, previous_position, last_movement_time)
                
                if hand_still:
                    hand_ready_for_gesture = True
                if hand_ready_for_gesture and not fist_detected:
                    last_fist_time = time.time()
                    fist_detected = True
                elif fist_detected and time.time() - last_fist_time > 1:
                    last_movement_time = time.time()
                    send_command_thread("pause")
                    command_last_time = time.time()
                    print("Fist closed: Pausing")
                    hand_ready_for_gesture = False
                    fist_detected = False
            else:
                fist_detected = False

            if is_palm_open(hand_landmarks.landmark, results.multi_handedness[0].classification[0].label[0] == "L"):
                hand_still, last_movement_time = is_hand_still(current_position, previous_position, last_movement_time)
                
                if hand_still:
                    hand_ready_for_gesture = True
                if hand_ready_for_gesture and not palm_detected:
                    last_palm_time = time.time()
                    palm_detected = True
                elif palm_detected and time.time() - last_palm_time > 1:
                    last_movement_time = time.time()
                    send_command_thread("play")
                    command_last_time = time.time()
                    print("Open palm: Playing")
                    hand_ready_for_gesture = False
                    palm_detected = False
            else:
                palm_detected = False

            if is_only_thumb_and_index_extended(hand_landmarks.landmark):
                volume = get_thumb_index_distance_action(hand_landmarks.landmark, last_volume_time)
                if volume == "volume up":
                    last_volume_time = time.time()
                    send_command_thread("volume_up")
                    command_last_time = time.time()
                elif volume == "volume down":
                    last_volume_time = time.time()
                    send_command_thread("volume_down")
                    command_last_time = time.time()
            

            previous_position = current_position
    elif hand_ready_for_swipe or hand_ready_for_gesture:
        hand_ready_for_swipe = False
        hand_ready_for_gesture = False
    
    img = Image.fromarray(frame_rgb)
    imgtk = ImageTk.PhotoImage(image=img)
    video_label.imgtk = imgtk
    video_label.config(image=imgtk)
    video_label.after(30, update_video)

# Start the video update loop for the webcam
update_video()

# Start the info panel update loop
update_info_panel()

update_command()

# Start the Tkinter event loop
root.mainloop()

# Release the webcam capture when the window is closed
cap.release()
